<center><img src="images/DLI_Header.png" alt="标题" style="width: 400px;"/></center>

# 3. 优化

目前，该模型遇到了[棋盘问题](https://distill.pub/2016/deconv-checkerboard/)。
<br/>
<center><img src="images/shoe_maybe.png" /></center>
<br/>
幸运的是，我们在生成的 T 恤袖子上有一些技巧可以解决这个问题，并普遍提高模型的性能。
<br/>
<br/>

#### 学习目标

本笔记本的目标是：
* 实现组标准化
* 实现 GELU
* 实现重新排列池化
* 实现正弦位置嵌入
* 定义反向扩散函数以模拟 `p`
* 尝试生成服装（再次）

像以前一样，让我们使用 fashionMIST 进行实验：

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import Adam

# Visualization tools
import matplotlib.pyplot as plt
from torchview import draw_graph
import graphviz
from IPython.display import Image

# User defined libraries
from utils import other_utils
from utils import ddpm_utils

IMG_SIZE = 16
IMG_CH = 1
BATCH_SIZE = 128
data, dataloader = other_utils.load_transformed_fashionMNIST(IMG_SIZE, BATCH_SIZE)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

我们创建了一个 [ddpm_util.py](utils/ddpm_utils.py)，其中包含一个 `DDPM` 类来对我们的扩散函数进行分组。让我们使用它来设置与之前使用的相同的 Beta 计划。

In [ ]:
nrows = 10
ncols = 15

T = nrows * ncols
B_start = 0.0001
B_end = 0.02
B = torch.linspace(B_start, B_end, T).to(device)
ddpm = ddpm_utils.DDPM(B, device)

## 3.1 组标准化和 GELU

我们首先要考虑的改进是优化标准卷积过程。我们将在整个神经网络中多次重复使用这个块，因此这是一个非常重要的部分。

### 3.1.1 组标准化

[批量标准化](https://towardsdatascience.com/batch-normalization-in-3-levels-of-understanding-14c2da90a338) 将每个内核通道的输出转换为 [z 分数](https://www.nlm.nih.gov/oet/ed/stats/02-910.html)。它通过计算一批输入的平均值和标准差来实现这一点。如果批量较小，这种方法是无效的。

另一方面，[组标准化](https://arxiv.org/pdf/1803.08494.pdf) 为每个样本图像标准化一组内核的输出，有效地“分组”一组特征。

<center><img src="images/groupnorm.png" /></center>

考虑到彩色图像有多个颜色通道，这会对生成图像的输出颜色产生有趣的影响。尝试实验看看效果！

欲了解有关规范化技术的更多信息，请参阅 Aakash Bindal 的这篇[博客文章](https://medium.com/techspace-usict/normalization-techniques-in-deep-neural-networks-9121bf100d8)。

### 3.1.2 GELU

[ReLU](https://www.kaggle.com/code/dansbecker/rectified-linear-units-relu-in-deep-learning) 是一种流行的激活函数选择，因为它计算速度快，而且计算梯度很容易。不幸的是，它并不完美。当偏差项变为很大程度的负值时，ReLU 神经元会[“死亡”](https://datascience.stackexchange.com/questions/5706/what-is-the-dying-relu-problem-in-neural-networks)，因为它的输出和梯度都为零。

[GELU](https://arxiv.org/pdf/1606.08415.pdf) 会稍微牺牲一点计算能力，试图通过模仿 ReLU 函数的形状来纠正整流线性单元，同时避免零梯度。

在这个 FashionMNIST 的小例子中，我们不太可能看到任何死亡神经元。然而，模型越大，越有可能面临 ReLU 消亡现象。

In [ ]:
class GELUConvBlock(nn.Module):
    def __init__(
        self, in_ch, out_ch, group_size):
        super().__init__()
        layers = [
            nn.Conv2d(in_ch, out_ch, 3, 1, 1),
            nn.GroupNorm(group_size, out_ch),
            nn.GELU()
        ]
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

## 3.2 重新排列池化

在之前的笔记本中，我们使用了 [Max Pooling](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html) 将我们的潜像大小减半，但这是最好的技术吗？有 [许多类型的池化层](https://pytorch.org/docs/stable/nn.html#pooling-layers)，包括最小池化和平均池化。我们让神经网络决定什么是重要的。

输入 [einops](https://einops.rocks/1-einops-basics/) 库和 [Rearrange](https://einops.rocks/api/rearrange/) 层。我们可以为每个层分配一个变量并使用它来重新排列我们的值。此外，我们可以使用括号 `()` 来标识一组相乘的变量。

例如，在下面的代码块中，我们有：

`Rearrange("b c (h p1) (w p2) -> b (c p1 p2) h w", p1=2, p2=2)`

* `b` 是我们的批处理维度
* `c` 是我们的通道维度
* `h` 是我们的高度维度
* `w` 是我们的宽度维度

我们还有一个 `p1` 和 `p2` 值，它们都等于 `2`。箭头前等式左侧部分表示“将高度和宽度维度分成两半”。箭头后等式右侧部分表示“沿通道维度堆叠分割维度”。

下面的代码块设置了一个 `test_image` 来练习。尝试在箭头左侧将 `h` 与 `p1` 交换。会发生什么？当 `w` 和 `p2` 交换时会怎样？当 `p1` 设置为 `3` 而不是 `2` 时会发生什么？

In [ ]:
from einops.layers.torch import Rearrange

rearrange = Rearrange("b c (h p1) (w p2) -> b (c p1 p2) h w", p1=2, p2=2)

test_image = [
    [
        [
            [1, 2, 3, 4, 5, 6],
            [7, 8, 9, 10, 11, 12],
            [13, 14, 15, 16, 17, 18],
            [19, 20, 21, 22, 23, 24],
            [25, 26, 27, 28, 29, 30],
            [31, 32, 33, 34, 35, 36],
        ]
    ]
]
test_image = torch.tensor(test_image)
print(test_image)
output = rearrange(test_image)
output

接下来，我们可以将其传递给我们的 `GELUConvBlock` ，让神经网络决定如何衡量“池”内的值。注意到 `GELUConvBlock` 的参数是 `4*in_chs` 吗？这是因为通道维度现在比 p1 * p2 大。

In [ ]:
class RearrangePoolBlock(nn.Module):
    def __init__(self, in_chs, group_size):
        super().__init__()
        self.rearrange = Rearrange("b c (h p1) (w p2) -> b (c p1 p2) h w", p1=2, p2=2)
        self.conv = GELUConvBlock(4 * in_chs, in_chs, group_size)

    def forward(self, x):
        x = self.rearrange(x)
        return self.conv(x)

我们现在有了重新定义 `DownBlock` 和 `UpBlock` 的组件。已添加多个 `GELUConvBlock` 来帮助解决棋盘问题。

In [ ]:
class DownBlock(nn.Module):
    def __init__(self, in_chs, out_chs, group_size):
        super(DownBlock, self).__init__()
        layers = [
            GELUConvBlock(in_chs, out_chs, group_size),
            GELUConvBlock(out_chs, out_chs, group_size),
            RearrangePoolBlock(out_chs, group_size)
        ]
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

**TODO**：`UpBlock` 有一个输入，将其与 `DownBlock` 分开。那是什么？

如果需要，请单击下面的 `...` 获取正确答案。

In [ ]:
class UpBlock(nn.Module):
    def __init__(self, in_chs, out_chs, group_size):
        super(UpBlock, self).__init__()
        layers = [
            nn.ConvTranspose2d(2 * in_chs, out_chs, 2, 2),
            GELUConvBlock(out_chs, out_chs, group_size),
            GELUConvBlock(out_chs, out_chs, group_size),
            GELUConvBlock(out_chs, out_chs, group_size),
            GELUConvBlock(out_chs, out_chs, group_size)
        ]
        self.model = nn.Sequential(*layers)

    def forward(self, x, FIXME):
        x = FIXME
        x = FIXME
        return x

In [ ]:
class UpBlock(nn.Module):
    def __init__(self, in_chs, out_chs, group_size):
        super(UpBlock, self).__init__()
        layers = [
            nn.ConvTranspose2d(2 * in_chs, out_chs, 2, 2),
            GELUConvBlock(out_chs, out_chs, group_size),
            GELUConvBlock(out_chs, out_chs, group_size),
            GELUConvBlock(out_chs, out_chs, group_size),
            GELUConvBlock(out_chs, out_chs, group_size),
        ]
        self.model = nn.Sequential(*layers)

    def forward(self, x, skip):
        x = torch.cat((x, skip), 1)
        x = self.model(x)
        return x

## 3.3 时间嵌入

模型对逆扩散过程的时间步长的理解越好，它就越能正确识别增加的噪声。在之前的笔记本中，我们为 `t/T` 创建了一个嵌入。我们能帮助模型更好地解释这一点吗？

在扩散模型出现之前，这是一个困扰自然语言处理的问题。对于长对话，我们如何才能捕捉到我们所处的位置？目标是找到一种方法，用少量的连续数字唯一地表示大量离散数字。使用单个浮点数是无效的，因为神经网络会将时间步长解释为连续的而不是离散的。[研究人员](https://arxiv.org/pdf/1706.03762.pdf)最终确定了正弦和余弦的总和。

有关其工作原理以及该技术可能如何开发的详细解释，请参阅 Jonathan Kernes 的 [Master Positional Encoding](https://towardsdatascience.com/master-positional-encoding-part-i-63c05d90a0c3)。

In [ ]:
import math

class SinusoidalPositionEmbedBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, time):
        device = time.device
        half_dim = self.dim // 2
        embeddings = math.log(10000) / (half_dim - 1)
        embeddings = torch.exp(torch.arange(half_dim, device=device) * -embeddings)
        embeddings = time[:, None] * embeddings[None, :]
        embeddings = torch.cat((embeddings.sin(), embeddings.cos()), dim=-1)
        return embeddings

**TODO**：我们将把 `SinusoidalPositionEmbedBlock` 的输出输入到我们的 `EmbedBlock` 中。幸运的是，我们的 `EmbedBlock` 与之前保持不变。

看起来下面的代码已经被 `FIXME` 覆盖了。你还记得它应该是什么样子吗？

如果需要，请单击下面的 `...` 获取正确答案。

In [ ]:
class EmbedBlock(nn.Module):
    def __init__(self, input_dim, emb_dim):
        super(EmbedBlock, self).__init__()
        self.input_dim = input_dim
        layers = [
            nn.Linear(input_dim, FIXME),
            nn.GELU(),
            nn.Linear(emb_dim, FIXME),
            nn.Unflatten(1, (FIXME, 1, 1))
        ]
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(-1, self.input_dim)
        return self.model(x)

In [ ]:
class EmbedBlock(nn.Module):
    def __init__(self, input_dim, emb_dim):
        super(EmbedBlock, self).__init__()
        self.input_dim = input_dim
        layers = [
            nn.Linear(input_dim, emb_dim),
            nn.GELU(),
            nn.Linear(emb_dim, emb_dim),
            nn.Unflatten(1, (emb_dim, 1, 1))
        ]
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(-1, self.input_dim)
        return self.model(x)

## 3.4 残差连接

消除棋盘问题的最后一个技巧是添加更多残差或跳过连接。我们可以为初始卷积创建一个 `ResidualConvBlock` 。我们也可以在其他地方添加残差连接，例如在我们的 `DownBlocks` 和 `UpBlocks` 中。

In [ ]:
class ResidualConvBlock(nn.Module):
    def __init__(self, in_chs, out_chs, group_size):
        super().__init__()
        self.conv1 = GELUConvBlock(in_chs, out_chs, group_size)
        self.conv2 = GELUConvBlock(out_chs, out_chs, group_size)

    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv2(x1)
        out = x1 + x2
        return out

下面是更新后的模型。注意到最后一行的变化了吗？从我们的 `ResidualConvBlock` 的输出到最后的 `self.out` 块添加了另一个跳跃连接。这个连接非常强大，在上面列出的所有变化中，它对该数据集的棋盘问题影响最大。

**TODO**：添加了几个新变量：`small_group_size` 和 `big_group_size` 用于组规范化。它们都依赖于变量 `group_base_size` 。将 `group_base_size` 设置为 `3` 、`4` 、`5` 、`6` 或 `7` 。其中一个值是正确的，其余的将导致错误。

**提示**：组大小和 `down_chs` 是相关的。

如果需要，请单击下面的 `...` 以获取正确答案。

In [ ]:
class UNet(nn.Module):
    def __init__(self):
        super().__init__()
        img_chs = IMG_CH
        down_chs = (64, 64, 128)
        up_chs = down_chs[::-1]  # Reverse of the down channels
        latent_image_size = IMG_SIZE // 4 # 2 ** (len(down_chs) - 1)
        t_dim = 8
        group_size_base = FIXME
        small_group_size = 2 * group_size_base  # New
        big_group_size = 8 * group_size_base  # New
        

        # Inital convolution
        self.down0 = ResidualConvBlock(img_chs, down_chs[0], small_group_size) # New

        # Downsample
        self.down1 = DownBlock(down_chs[0], down_chs[1], big_group_size) # New
        self.down2 = DownBlock(down_chs[1], down_chs[2], big_group_size) # New
        self.to_vec = nn.Sequential(nn.Flatten(), nn.GELU())
        
        # Embeddings
        self.dense_emb = nn.Sequential(
            nn.Linear(down_chs[2]*latent_image_size**2, down_chs[1]),
            nn.ReLU(),
            nn.Linear(down_chs[1], down_chs[1]),
            nn.ReLU(),
            nn.Linear(down_chs[1], down_chs[2]*latent_image_size**2),
            nn.ReLU()
        )

        self.sinusoidaltime = SinusoidalPositionEmbedBlock(t_dim) # New
        self.temb_1 = EmbedBlock(t_dim, up_chs[0])
        self.temb_2 = EmbedBlock(t_dim, up_chs[1])
        
        # Upsample
        self.up0 = nn.Sequential(
            nn.Unflatten(1, (up_chs[0], latent_image_size, latent_image_size)),
            GELUConvBlock(up_chs[0], up_chs[0], big_group_size) # New
        )
        self.up1 = UpBlock(up_chs[0], up_chs[1], big_group_size) # New
        self.up2 = UpBlock(up_chs[1], up_chs[2], big_group_size) # New

        # Match output channels and one last concatenation
        self.out = nn.Sequential(
            nn.Conv2d(2 * up_chs[-1], up_chs[-1], 3, 1, 1),
            nn.GroupNorm(small_group_size, up_chs[-1]), # New
            nn.ReLU(),
            nn.Conv2d(up_chs[-1], img_chs, 3, 1, 1)
        )

    def forward(self, x, t):
        down0 = self.down0(x)
        down1 = self.down1(down0)
        down2 = self.down2(down1)
        latent_vec = self.to_vec(down2)
        
        latent_vec = self.dense_emb(latent_vec)
        t = t.float() / T  # Convert from [0, T] to [0, 1]
        t = self.sinusoidaltime(t) # New
        temb_1 = self.temb_1(t)
        temb_2 = self.temb_2(t)

        up0 = self.up0(latent_vec)
        up1 = self.up1(up0+temb_1, down2)
        up2 = self.up2(up1+temb_2, down1)
        return self.out(torch.cat((up2, down0), 1)) # New

In [ ]:
class UNet(nn.Module):
    def __init__(self):
        super().__init__()
        img_chs = IMG_CH
        down_chs = (64, 64, 128)
        up_chs = down_chs[::-1]  # Reverse of the down channels
        latent_image_size = IMG_SIZE // 4 # 2 ** (len(down_chs) - 1)
        t_dim = 8
        group_size_base = 4
        small_group_size = 2 * group_size_base # New
        big_group_size = 8 * group_size_base  # New
        

        # Inital convolution
        self.down0 = ResidualConvBlock(img_chs, down_chs[0], small_group_size) # New

        # Downsample
        self.down1 = DownBlock(down_chs[0], down_chs[1], big_group_size) # New
        self.down2 = DownBlock(down_chs[1], down_chs[2], big_group_size) # New
        self.to_vec = nn.Sequential(nn.Flatten(), nn.GELU())
        
        # Embeddings
        self.dense_emb = nn.Sequential(
            nn.Linear(down_chs[2]*latent_image_size**2, down_chs[1]),
            nn.ReLU(),
            nn.Linear(down_chs[1], down_chs[1]),
            nn.ReLU(),
            nn.Linear(down_chs[1], down_chs[2]*latent_image_size**2),
            nn.ReLU()
        )

        self.sinusoidaltime = SinusoidalPositionEmbedBlock(t_dim) # New
        self.temb_1 = EmbedBlock(t_dim, up_chs[0])
        self.temb_2 = EmbedBlock(t_dim, up_chs[1])
        
        # Upsample
        self.up0 = nn.Sequential(
            nn.Unflatten(1, (up_chs[0], latent_image_size, latent_image_size)),
            GELUConvBlock(up_chs[0], up_chs[0], big_group_size) # New
        )
        self.up1 = UpBlock(up_chs[0], up_chs[1], big_group_size) # New
        self.up2 = UpBlock(up_chs[1], up_chs[2], big_group_size) # New

        # Match output channels and one last concatenation
        self.out = nn.Sequential(
            nn.Conv2d(2 * up_chs[-1], up_chs[-1], 3, 1, 1),
            nn.GroupNorm(small_group_size, up_chs[-1]), # New
            nn.ReLU(),
            nn.Conv2d(up_chs[-1], img_chs, 3, 1, 1)
        )

    def forward(self, x, t):
        down0 = self.down0(x)
        down1 = self.down1(down0)
        down2 = self.down2(down1)
        latent_vec = self.to_vec(down2)
        
        latent_vec = self.dense_emb(latent_vec)
        t = t.float() / T  # Convert from [0, T] to [0, 1]
        t = self.sinusoidaltime(t) # New
        temb_1 = self.temb_1(t)
        temb_2 = self.temb_2(t)

        up0 = self.up0(latent_vec)
        up1 = self.up1(up0+temb_1, down2)
        up2 = self.up2(up1+temb_2, down1)
        return self.out(torch.cat((up2, down0), 1)) # New

In [ ]:
model = UNet()
print("Num params: ", sum(p.numel() for p in model.parameters()))
model = torch.compile(model.to(device))

最后，是时候训练模型了。让我们看看所有这些变化是否产生了影响。

In [ ]:
optimizer = Adam(model.parameters(), lr=0.001)
epochs = 5

model.train()
for epoch in range(epochs):
    for step, batch in enumerate(dataloader):
        optimizer.zero_grad()

        t = torch.randint(0, T, (BATCH_SIZE,), device=device).float()
        x = batch[0].to(device)
        loss = ddpm.get_loss(model, x, t)
        loss.backward()
        optimizer.step()

        if epoch % 1 == 0 and step % 100 == 0:
            print(f"Epoch {epoch} | step {step:03d} Loss: {loss.item()} ")
            ddpm.sample_images(model, IMG_CH, IMG_SIZE, ncols)

仔细看看怎么样？你能认出是鞋子、钱包还是衬衫吗？

In [ ]:
model.eval()
plt.figure(figsize=(8,8))
ncols = 3 # Should evenly divide T
for _ in range(10):
    ddpm.sample_images(model, IMG_CH, IMG_SIZE, ncols)

## 3.5 下一步

如果您没有看到鞋子或衬衫等特定类别，请尝试再次运行上述代码块。目前，我们的模型不接受类别输入，因此用户无法定义他们想要什么样的输出。这有什么乐趣呢？

在下一个笔记本中，我们将为用户添加一种控制模型的方法！

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<center><img src="images/DLI_Header.png" alt="标题" style="width: 400px;"/></center>